# PIXELSOUNDS
## Codificador de audio a vídeo
### Introducción

En este caso, y siguiendo en la línea de los anteriores conversores propuestos en el proyecto PixelSounds, vamos a tratar de convertir audio a vídeo. Sin embargo, no va a ser cualquier tipo de vídeo. Vamos a tratar de generar un vídeo que nos permita visualizar en una primera vista algunas características del audio que hay detrás. Para ello vamos a extraer algunos descriptores del audio (los cuales especificaremos más adelante) y vamos a transformarlos en algo visual. De esta forma, iremos generando fotogramas y mediante la libreria `ffmpeg` los uniremos para crear un video. En este cuaderno vamos a ir desgranando una por una las transformaciones que haremos para visualizar los distintos descriptores. Después uniremos todas las funciones para generar un fotograma. Tras aprender a generar un fotograma, podremos generar muchos para crear el vídeo.

### Librerias necesarias

In [1]:
import vozyaudio as vz
import numpy as np
import matplotlib.pyplot as plt
import subprocess
import os
import shutil
from vozyaudio import lee_audio, envolvente, track_pitch, espectro
from scipy.signal import resample, correlate, find_peaks

### Los primeros pasos
Antes de comenzar a hacer nada, vamos a definir algunos parámetros básicos que vamos a utilizar más adelante. La configuración del codificador consta de los siguientes parámetros.

In [2]:
# CONFIGURACIÓN
AUDIO_PATH = 'audios/music.wav' # Ruta del audio que vamos a usar
FPS = 25 # Número de fotogramas por segundo del vídeo resultante
FRAME_FOLDER = 'fotogramas' # Ruta de la carpeta donde iremos guardando los fotogramas
VIDEO_PATH = 'output.mp4' # Nombre del video
N_BARRAS = 60  # Número de barras del espectro

Después de esto, cargaremos el audio y extraeremos su duración, número de muestras, y muestras por fotograma. Además, obtendremos el número de frames que ha de tener el vídeo resultante

In [3]:
# Cargar audio
fs, x = lee_audio(AUDIO_PATH)
x = x.astype(np.float32)
dur = len(x) / fs
n_frames = int(FPS * dur)
samples_per_frame = int(fs / FPS)

Tras hacer todo esto ya podemos empezar a generar las primeras componentes del fotograma

### Círculo que se mueve con el pitch

Lo primero que vamos a agregar al fotograma es un **circulo que se mueve de arriba a abajo con el pitch y cambia de tamaño con la envolvente**. Para este paso, usaremos las funciones `envolvente` y `track_pitch` del módulo `vozyaudio` para extraer la envolvente y estimar el pitch del audio. Despues normalizaremos ambos descriptores y los redimensionaremos para que se ajusten al número de frames finales del video

In [4]:
# Extraemos los descriptores de envolvente y estimación de pitch
env = envolvente(x, fs=fs) # Extraer envolvente
pitch = track_pitch(x, fs) # Estimar pitch
pitch = np.nan_to_num(pitch)  # Reemplaza NaNs por 0

def normalizar(v):
    """ Normaliza un vector al rango [0, 1].
    Entrada:
        v (numpy.ndarray): Vector de valores (por ejemplo, envolvente, pitch, espectro, etc.)
    Salida:
        v_norm (numpy.ndarray): Vector normalizado en el rango [0, 1]
    """
    return (v - np.min(v)) / (np.max(v) - np.min(v) + 1e-9)

env = normalizar(env) # Normalizar ambos arrays
pitch = normalizar(pitch)

# Redimensionar pitch y envolvente al número de frames
pitch_frame = np.interp(np.linspace(0, len(pitch), n_frames), np.arange(len(pitch)), pitch)
env_frame = np.interp(np.linspace(0, len(env), n_frames), np.arange(len(env)), env)

Con estas variables ya tenemos todo lo necesario para generar el primer componente que variará durante el vídeo. Lo siguiente que debemos hacer es definir una función que dibujo el circulo en pantalla variando segúne estos valores. A esta función la llamaremos `dibujar_particula`.

In [5]:
def dibujar_particula(ax, pitch, env):
    """ Dibuja una partícula que se mueve según el pitch y cambia de tamaño según la envolvente del audio
    Entrada:
        ax (matplotlib.axes.Axes): Objeto de ejes sobre el que se dibuja la partícula
        pitch (numpy.ndarray) : Array con los valores de estimación del pitch del audio
        env (numpy.ndarray) : Array con los valores de la envolvente del audio
    Salida:
        None: La función solo dibuja en la figura y no devuelve nada
    """
    y_pos = pitch
    size = 100 + env * 300
    color = (1.0, env, pitch)
    ax.scatter(0.5, y_pos, s=size, c=[color], alpha=0.8)

Para visualizar que el resultado es el que esperamos tenemos que definir un esqueleto básico de generación de frames. Este esqueleto lo iremos ampliando a medida que vayamos añadiendo más componentes al fotograma. Cuando tengamos todos los fotogramas generados los uniremos mediante la función `crear_video`  para ver el resultado. Esta función hace uso de `subprocess` y de `ffmpeg` para crear el video de manera rápida simplemente pasándole el archivo `generarVideo.bat` que se encargará de ejecutar los comandos ffmpeg. La usaremos mucho a lo largo del cuaderno. De momento, probemos a generar los fotogramas con solo esta componente y ver el video resultado.

In [ ]:
# Generar frames
print("Generando frames...")
for i in range(n_frames):
    porcentaje = (i / n_frames) * 100
    print(f"\rCompletado {porcentaje:.2f} %", end="", flush=True)
    
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.set_facecolor((0, 0, 0))  # Fondo negro

    # Visual: Círculo que sube/baja con pitch y cambia tamaño con envolvente
    dibujar_particula(ax, pitch_frame[i], env_frame[i])

    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')

    plt.tight_layout()
    plt.savefig(f"{FRAME_FOLDER}/frame_{i:04d}.png")
    plt.close(fig)

print("Frames generados.")

try:
    subprocess.run(['generarVideo.bat', AUDIO_PATH], check=True)
except subprocess.CalledProcessError as e:
    print("Error al ejecutar generarVideo.bat:", e)
finally:
    # shutil.rmtree('fotogramas/')
    print('Procesado terminado.')

Generando frames...
Frames generados.


In [4]:
def normalizar(v):
    """ Normaliza un vector al rango [0, 1].
    Entrada:
        v (numpy.ndarray): Vector de valores (por ejemplo, envolvente, pitch, espectro, etc.)
    Salida:
        v_norm (numpy.ndarray): Vector normalizado en el rango [0, 1]
    """
    return (v - np.min(v)) / (np.max(v) - np.min(v) + 1e-9)


def autocorrelacion(x_frame):
    """ Calcula la autocorrelación normalizada de una ventana de señal de audio.
    Entrada:
        x_frame (numpy.ndarray): Fragmento de señal de audio (ventana temporal)
    Salida:
        corr_norm (numpy.ndarray): Autocorrelación normalizada desde el retardo cero hacia adelante
    """
    x_frame = x_frame - np.mean(x_frame)
    corr = correlate(x_frame, x_frame, mode='full')
    mid = len(corr) // 2
    return corr[mid:] / np.max(np.abs(corr) + 1e-9)


def detectar_ritmo(x_frame, fs, fmin=1.5, fmax=8):
    """ Estima el periodo rítmico de un fragmento de audio mediante autocorrelación.
    Entrada:
        x_frame (numpy.ndarray): Fragmento de señal de audio (ventana temporal)
        fs (int): Frecuencia de muestreo del audio
        fmin (float): Frecuencia mínima esperada del ritmo (en Hz)
        fmax (float): Frecuencia máxima esperada del ritmo (en Hz)
    Salida:
        periodo_seg (float): Periodo estimado del ritmo en segundos
        corr (numpy.ndarray): Autocorrelación normalizada del fragmento de audio
    """
    corr = autocorrelacion(x_frame)
    min_lag = int(fs / fmax)
    max_lag = int(fs / fmin)
    if max_lag >= len(corr): max_lag = len(corr) - 1
    if min_lag >= max_lag: return 0.5, corr  # Valor por defecto
    pico = np.argmax(corr[min_lag:max_lag]) + min_lag
    periodo_seg = pico / fs
    return periodo_seg, corr


In [ ]:
# === 1. Cargar audio ===
fs, x = lee_audio(AUDIO_PATH)
x = x.astype(np.float32)
dur = len(x) / fs
n_frames = int(FPS * dur)
samples_per_frame = int(fs / FPS)

# === 2. Descriptores ===
env = envolvente(x, fs=fs)
pitch = track_pitch(x, fs)
pitch = np.nan_to_num(pitch)


In [ ]:
# 1. Autocorrelación sobre la envolvente
env_smooth = envolvente(x, fs=fs, tr=0.1)  # más estable
corr_env = autocorrelacion(env_smooth)

# 2. Estimar el tempo global
min_lag = int(fs / 5)    # máx 5 Hz = 300 BPM
max_lag = int(fs / 1.5)  # mín 1.5 Hz = 90 BPM
lag_beat = np.argmax(corr_env[min_lag:max_lag]) + min_lag
periodo_muestras = lag_beat

# 3. Encontrar los picos en la envolvente
peaks, _ = find_peaks(env_smooth, distance=periodo_muestras * 0.8)

# Convertir los picos (en muestras) a tiempos (en segundos) y luego a frames
beat_times = peaks / fs
beat_frames = (beat_times * FPS).astype(int)


In [ ]:
env = normalizar(env)
pitch = normalizar(pitch)

# Redimensionar descriptores al número de frames
env_frame = np.interp(np.linspace(0, len(env), n_frames), np.arange(len(env)), env)
pitch_frame = np.interp(np.linspace(0, len(pitch), n_frames), np.arange(len(pitch)), pitch)


In [ ]:
def es_beat(frame_index, beat_frames, tolerancia=2):
    """ Determina si un frame está dentro de los limites de un beat detectado.
    Entrada:
        frame_index (int): Índice del frame actual en el video
        beat_frames (list of int): Lista de frames donde se detectaron beats
        tolerancia (int): Número de frames de margen alrededor de cada beat
    Salida:
        es_beat (bool): True si el frame está cerca de un beat, False en caso contrario
    """
    return any(abs(frame_index - bf) <= tolerancia for bf in beat_frames)


def dibujar_flash(ax):
    """ Dibuja un flash visual en el centro del frame, usado para resaltar beats detectados.
    Entrada:
        ax (matplotlib.axes.Axes): Objeto de ejes sobre el que se dibuja el flash
    Salida:
        None: El flash se dibuja directamente sobre la figura
    """
    ax.scatter(0.5, 0.5, s=1500, c='cyan', alpha=0.9, edgecolors='none', marker='o')


def obtener_frame_audio(x, i, samples_per_frame):
    """ Extrae un fragmento de audio correspondiente a un frame de video.
    Entrada:
        x (numpy.ndarray): Señal de audio completa
        i (int): Índice del frame actual
        samples_per_frame (int): Número de muestras de audio por frame
    Salida:
        x_frame (numpy.ndarray): Fragmento de audio correspondiente al frame i
    """
    start = i * samples_per_frame
    end = min(len(x), start + samples_per_frame)
    return x[start:end]


def dibujar_circulo_ritmico(ax, t_actual, periodo):
    """ Dibuja un círculo que pulsa rítmicamente en el centro del frame según un periodo dado.
    Entrada:
        ax (matplotlib.axes.Axes): Objeto de ejes sobre el que se dibuja el círculo
        t_actual (float): Tiempo actual del video en segundos
        periodo (float): Periodo rítmico estimado en segundos
    Salida:
        None: El círculo se dibuja directamente sobre la figura
    """
    ritmo_osc = 0.5 * (1 + np.sin(2 * np.pi * t_actual / periodo))
    color = (ritmo_osc, 0.2, 1 - ritmo_osc)
    size = 300 * ritmo_osc + 20
    ax.scatter(0.5, 0.5, s=size, c=[color], alpha=0.3)


def dibujar_barras(ax, X_resampled, N_BARRAS):
    """ Dibuja barras verticales que representan la energía en diferentes bandas de frecuencia.
    Entrada:
        ax (matplotlib.axes.Axes): Objeto de ejes sobre el que se dibujan las barras
        X_resampled (numpy.ndarray): Vector con las amplitudes espectrales reescaladas a N_BARRAS bandas
        N_BARRAS (int): Número total de barras a dibujar
    Salida:
        None: Las barras se dibujan directamente sobre la figura
    """
    bar_width = 1 / N_BARRAS
    for j in range(N_BARRAS):
        height = X_resampled[j]
        color = (0.1, 0.8 * height, 1.0)
        ax.bar(j * bar_width, height, width=bar_width*0.8, color=color, align='edge')

def dibujar_particula(ax, pitch, env):
    y_pos = pitch
    size = 100 + env * 300
    color = (1.0, env, pitch)
    ax.scatter(0.5, y_pos, s=size, c=[color], alpha=0.8)

def finalizar_figura(fig, ax, path):
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')
    plt.tight_layout()
    plt.savefig(path)
    plt.close(fig)


In [ ]:
def generar_frames(x, fs, pitch_frame, env_frame, beat_frames, n_frames, samples_per_frame, FPS, N_BARRAS, FRAME_FOLDER):
    print("Generando frames...")
    
    # Crear carpeta
    os.makedirs(FRAME_FOLDER, exist_ok=True)
    
    for i in range(n_frames):
        porcentaje = (i / n_frames) * 100

        print(f"\rCompletado {porcentaje:.2f} %", end="", flush=True)
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.set_facecolor((0, 0, 0))
        
        if es_beat(i, beat_frames):
            dibujar_flash(ax)
        
        x_frame = obtener_frame_audio(x, i, samples_per_frame)
        t_actual = i / FPS
        periodo, _ = detectar_ritmo(x_frame, fs)
        dibujar_circulo_ritmico(ax, t_actual, periodo)

        X, _ = espectro(x_frame, modo=1, fs=fs)
        X_resampled = normalizar(resample(X, N_BARRAS))
        dibujar_barras(ax, X_resampled, N_BARRAS)

        dibujar_particula(ax, pitch_frame[i], env_frame[i])
        
        finalizar_figura(fig, ax, f"{FRAME_FOLDER}/frame_{i:04d}.png")
    print("\nFrames generados.")


In [ ]:
# === 3. Generar Frames ===
print("Generando frames...")
for i in range(n_frames):
    porcentaje = (i / n_frames) * 100
    print(f"\rCompletado {porcentaje:.2f} %", end="", flush=True)
        
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.set_facecolor((0, 0, 0))  # Fondo negro
    
    # Flash más visible en beat
    if any(abs(i - bf) <= 2 for bf in beat_frames):  # mayor tolerancia
        ax.scatter(0.5, 0.5, s=1500, c='cyan', alpha=0.9, edgecolors='none', marker='o')

    # Halo pulsante animado en el centro tras beat
    for bf in beat_frames:
        frames_from_beat = i - bf
        if 0 <= frames_from_beat <= 4:  # duración 4 frames
            grow = 1 - frames_from_beat / 4
            size = 2000 * grow
            alpha = 0.8 * grow
            ax.scatter(0.5, 0.5, s=size, c='magenta', alpha=alpha, edgecolors='none')

    # ==== 3.1 Obtener trozo de señal actual ====
    start = i * samples_per_frame
    end = min(len(x), start + samples_per_frame)
    x_frame = x[start:end]
    
    # Detectar ritmo
    periodo, _ = detectar_ritmo(x_frame, fs)
    t_actual = i / FPS
    ritmo_osc = 0.5 * (1 + np.sin(2 * np.pi * t_actual / periodo))  # 0..1

    # Efecto visual rítmico: círculo que late en el centro
    ritmo_color = (ritmo_osc, 0.2, 1 - ritmo_osc)
    ritmo_size = 300 * ritmo_osc + 20
    ax.scatter(0.5, 0.5, s=ritmo_size, c=[ritmo_color], alpha=0.3)
    
    # ==== 3.2 Espectro (resample a N barras) ====
    X, fa = espectro(x_frame, modo=1, fs=fs)
    X_resampled = resample(X, N_BARRAS)
    X_resampled = normalizar(X_resampled)

    # ==== 3.3 Dibujar barras ====
    bar_width = 1 / N_BARRAS
    for j in range(N_BARRAS):
        height = X_resampled[j]
        ax.bar(j * bar_width, height, width=bar_width*0.8, color=(0.1, 0.8*height, 1.0), align='edge')

    # ==== 3.4 Dibujar partícula ====
    y_pos = pitch_frame[i]
    size = 100 + env_frame[i] * 300
    color = (1.0, env_frame[i], pitch_frame[i])
    ax.scatter(0.5, y_pos, s=size, c=[color], alpha=0.8)

    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')
    plt.tight_layout()
    plt.savefig(f"{FRAME_FOLDER}/frame_{i:04d}.png")
    plt.close(fig)

print("Frames generados.")


In [ ]:
# Generar Video
try:
    subprocess.run(['generarVideo.bat', AUDIO_PATH], check=True)
except subprocess.CalledProcessError as e:
    print("Error al ejecutar generarVideo.bat:", e)
finally:
    # shutil.rmtree('fotogramas/')

    print('Procesado terminado.')